In [13]:
import win32com.client
import os
from find_buttons import find_buttons
from find_text_v2 import find_text
from langdetect import detect as detect_lang, detect_langs
import re
from photoshop import Session
import photoshop.api as ps
import keyboard
import time
import subprocess
from keyHooks import hook, unhook, loop, wait
import winsound

In [23]:
app = win32com.client.Dispatch('Photoshop.Application')
app.Open(r"C:\Users\dahom\Desktop\ps_screen\Project.psd")
doc = app.Application.ActiveDocument

In [14]:
def mngButtons(filename):
    lit_buttons = find_buttons(filename)
    for i in range(1,8+1):
        layer = doc.LayerSets("Buttons").Layers("btn"+str(i))
        if(i in lit_buttons):
            layer.visible = True
        else:
            layer.visible = False

In [15]:
def mngText(filename):
    data = find_text(filename)
    #print(data)
    def mngButtonText():
        for btn in data['buttons']:
            text = data['buttons'][btn]['text']
            editButton(btn, text)
    def mngNonButtonText():
        floatingGrp = doc.LayerSets("Buttons").Layers("floatingText").ArtLayers
        try:
            floatingGrp.RemoveAll()
        except:
            pass

        for group in data['nonbuttons']:
            text = data['nonbuttons'][group]['text']
            pos = data['nonbuttons'][group]['pos']
            createText(text, pos)

    mngNonButtonText()  
    mngButtonText()

In [16]:
def getLang(text):
    res_lang = ''
    i = 0
    try:
        langs = detect_langs(text)
        for lang in langs:
            if(lang.lang in ['ar','eng']):
                if(lang.prob > i):
                    i = lang.prob
                    res_lang = lang.lang
                    res_lang = 'ara' if res_lang == 'ar' else res_lang
    except:
        pass
    if(res_lang == ''):
#      the detection above failed, try character approch
        if (re.search('[a-zA-Z]', text)):
            return 'eng'
        else:
            return 'ara'
    return res_lang


In [17]:
def editButton(num, newText):
    # edit a specific buttons text
    lang = getLang(newText)
    #print('btn{}.lang({})'.format(str(num), lang))
    layer = doc.LayerSets("Buttons").Layers("btn"+str(num)).Layers(lang)
    layer.visible = True
    text_of_layer = layer.TextItem
    text_of_layer.contents = newText
    #  hide the other text?
    doc.LayerSets("Buttons").Layers("btn"+str(num)).Layers('ara' if lang == 'eng' else 'eng').visible = False


In [18]:
# AddText, disable else text
def createText(text, pos):
    lang = getLang(text)
    layer = doc.LayerSets("Buttons").Layers("floatingText")
    layer = doc.ArtLayers.Add()
    layer.kind = 2 #TEXT
    layer.textItem.contents = text
    layer.textItem.position = [pos[0], pos[1]]
    layer.textItem.size = 13 if lang == 'eng' else 14
    layer.textItem.Font = 'AvantGardeITCbyBT-Medium' if lang == 'eng' else 'GESSTextMedium-Medium'
    layer.textItem.Justification = 3 if lang == 'eng' else 1
    # Color???
    layer.Move(doc.LayerSets("Buttons").LayerSets("floatingText"),1)
    return layer


In [8]:
# filename = 'ICON257.jpg'
# mngText(filename)
# mngButtons(filename)

In [19]:
def saveTo(path, method=2):
    if(method == 0): #PSD
#         doc.SaveAs(SaveIn=path, AsCopy=True, ExtensionType=2)
        options = win32com.client.Dispatch('Photoshop.PhotoshopSaveOptions')
        options.embedColorProfile = True;
        options.alphaChannels = True;
        doc.SaveAs(SaveIn=path, AsCopy=False, Options=options, ExtensionType=2)
    elif(method == 1):
        options = win32com.client.Dispatch('Photoshop.PNGSaveOptions')
        doc.SaveAs(SaveIn=path, Options=options, AsCopy=True, ExtensionType=2)
    else:
        # Very small Size compared to SaveAs
        options = win32com.client.Dispatch('Photoshop.ExportOptionsSaveForWeb')
        options.Format = 13
        options.PNG8 = False
        doc.Export(ExportIn=path, ExportAs=2, Options=options)

In [20]:
def do_action(filename):
    filePath = os.path.join(input_dir, filename)
    outFilePath = os.path.join(output_dir, filename)
    mngText(filePath)
    mngButtons(filePath)
   

In [21]:
input_dir = r'C:\Users\dahom\Desktop\ps_screen\input'
output_dir = r'C:\Users\dahom\Desktop\ps_screen\output'
psd_dir = r'C:\Users\dahom\Desktop\ps_screen\output\PSD'
def Main():
    for root, directories, files in os.walk(input_dir, topdown=False):
        for name in files:
            file_path = os.path.join(root, name)
            file_outPath = os.path.join(output_dir, name)
            psd_outPath = os.path.join(psd_dir, name)
            print(f'Proccessing ({name})')
            process = subprocess.Popen([r"C:\Program Files\IrfanView\i_view64.exe", file_path])
            do_action(name)
            winsound.MessageBeep()
            print('\tWaiting Next, current not saved yet')
            while(True):
                if(keyboard.is_pressed("ctrl + f1")):
                    return
                if(keyboard.is_pressed("alt + j")):
                    time.sleep(0.4)
                    break
                time.sleep(0.2)
            print('\t Saving...')
            saveTo(file_outPath, 1)
            saveTo(psd_outPath, 0)
            print(f'\t done {name},'+'{}/{}'.format(len(os.listdir(input_dir)), len(os.listdir(output_dir))))
            process.kill()
            if os.path.exists(file_path):
                os.remove(file_path)
            else:
                print(f'Failed to remove {name}')

In [26]:
Main()
print('Done ALL')

Proccessing (ICON324.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON324.jpg,122/162
Proccessing (ICON325.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON325.jpg,121/163
Proccessing (ICON333.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON333.jpg,120/164
Proccessing (ICON334.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON334.jpg,119/165
Proccessing (ICON336.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON336.jpg,118/166
Proccessing (ICON337.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON337.jpg,117/167
Proccessing (ICON341.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON341.jpg,116/168
Proccessing (ICON344-.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON344-.jpg,115/169
Proccessing (ICON344.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done ICON344.jpg,114/170
Proccessing (ICON346.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 don

	Waiting Next, current not saved yet
	 Saving...
	 done pic025.jpg,41/243
Proccessing (pic029.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done pic029.jpg,40/244
Proccessing (Pic030.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done Pic030.jpg,39/245
Proccessing (PIC034.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done PIC034.jpg,38/246
Proccessing (pic047.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done pic047.jpg,37/247
Proccessing (pic100.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done pic100.jpg,36/248
Proccessing (pic102.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done pic102.jpg,35/249
Proccessing (pic105.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done pic105.jpg,34/250
Proccessing (pic106.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done pic106.jpg,33/251
Proccessing (PIC120.jpg)
	Waiting Next, current not saved yet
	 Saving...
	 done PIC120.jpg,32/252
Proccessing (Pic130.jpg)
	Waiting N